[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/FineTuning/blob/main/lessons/module3_advanced/02_multitask_learning.ipynb)

# 🎨 Multi-Task Learning: One Model, Many Jobs

**Duration:** 1.5 hours  
**Level:** Advanced  
**Prerequisites:** Module 2 complete

---

## Hey! Want Your Model to Do Multiple Things? 🤹

Imagine this:

**Right now, you probably think:**
- One model = One task
- Sentiment analysis model
- Summarization model
- Q&A model

**But what if ONE model could:**
- Analyze sentiment ✅
- AND summarize text ✅
- AND answer questions ✅
- AND classify topics ✅

**All at the same time?**

That's **Multi-Task Learning (MTL)**!

Think of it like a Swiss Army knife 🔪🔧✂️ - one tool, many functions!

Let me show you how to build one... 🚀

## 🤔 Why Multi-Task Learning?

**Benefits:**

### 1. **Efficiency** ⚡
- Train ONE model instead of 5 separate models
- Save training time
- Save GPU costs

### 2. **Better Performance** 📈
- Tasks help each other learn!
- Sentiment analysis helps with topic classification
- Q&A helps with summarization
- **Shared knowledge = Better results**

### 3. **Less Data Needed** 💾
- Each task's data helps the others
- Got lots of sentiment data but little summarization data?
- Perfect! The sentiment task helps the summarization task learn!

### 4. **Deployment** 🚀
- Deploy ONE model instead of multiple
- Less memory in production
- Simpler architecture

**Real Example:**
Google's BERT was trained on multiple tasks:
- Masked language modeling
- Next sentence prediction

That's why it works so well for everything!

## 🏗️ How Multi-Task Learning Works

Simple concept:

**Regular Training:**
```
Input → Shared Encoder → Task A Head → Output A
```

**Multi-Task Training:**
```
                    ┌→ Task A Head → Output A
                    │
Input → Shared Encoder ├→ Task B Head → Output B
                    │
                    └→ Task C Head → Output C
```

**The magic:**
- **Shared Encoder** learns features that help ALL tasks
- **Task-specific Heads** specialize for each task
- During training, we alternate between tasks

**Like learning multiple languages:**
- Grammar rules are shared (encoder)
- Vocabulary is specific (heads)
- Learning Spanish helps you learn Italian!

## 🛠️ Let's Build a Multi-Task Model!

We'll create a model that does:
1. **Sentiment Analysis** (positive/negative)
2. **Topic Classification** (tech/sports/politics/entertainment)
3. **Text Length Prediction** (short/medium/long)

Three tasks, one model! Let's go!

In [ ]:
!pip install -q transformers datasets torch accelerate evaluate scikit-learn

## Step 1: Create Our Multi-Task Model Architecture

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class MultiTaskModel(nn.Module):
    """One model, multiple tasks!"""
    
    def __init__(self, model_name, num_tasks_config):
        super().__init__()
        
        # Shared encoder - this learns features for ALL tasks
        print("🧠 Creating shared encoder...")
        self.encoder = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.encoder.config.hidden_size
        
        # Task-specific heads
        print("🎯 Creating task-specific heads...")
        self.task_heads = nn.ModuleDict()
        
        for task_name, num_labels in num_tasks_config.items():
            # Each task gets its own classification head
            self.task_heads[task_name] = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(self.hidden_size, num_labels)
            )
            print(f"   ✅ {task_name}: {num_labels} classes")
    
    def forward(self, input_ids, attention_mask, task_name):
        # Get shared representations
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0, :]
        
        # Route to appropriate task head
        logits = self.task_heads[task_name](pooled_output)
        
        return logits

# Define our tasks
tasks_config = {
    'sentiment': 2,      # positive/negative
    'topic': 4,          # tech/sports/politics/entertainment
    'length': 3,         # short/medium/long
}

print("\n🏗️ Building multi-task model...\n")
model = MultiTaskModel('distilbert-base-uncased', tasks_config)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

print("\n✅ Model ready!")
print(f"\n📊 Model size: {sum(p.numel() for p in model.parameters()):,} parameters")

## Step 2: Create Multi-Task Dataset

We need data for all three tasks. Let's create synthetic data for demo purposes:

In [ ]:
from datasets import Dataset
import random

# Create synthetic dataset
def create_multitask_dataset(num_samples=1000):
    """Generate synthetic data for our three tasks"""
    
    topics = ['tech', 'sports', 'politics', 'entertainment']
    
    # Templates for different topics and sentiments
    templates = {
        'tech': {
            'positive': [
                "This new smartphone is absolutely amazing! The features are incredible.",
                "I love this software update. It makes everything so much faster!",
                "Best laptop I've ever owned. The performance is outstanding."
            ],
            'negative': [
                "This app crashes constantly. Very frustrating experience.",
                "Terrible battery life on this device. Completely disappointing.",
                "The new update broke everything. This is unacceptable."
            ]
        },
        'sports': {
            'positive': [
                "What an incredible game! The team played brilliantly.",
                "Amazing performance by the athletes. They deserved this win!",
                "Best match of the season. Absolutely thrilling to watch."
            ],
            'negative': [
                "Terrible performance today. The team looked completely lost.",
                "Very disappointing game. They need to improve significantly.",
                "Worst match I've seen. Completely unorganized play."
            ]
        },
        'politics': {
            'positive': [
                "Great policy announcement today. This will help many people.",
                "Excellent speech by the leader. Very inspiring and hopeful.",
                "Important reforms being implemented. Positive changes ahead."
            ],
            'negative': [
                "Terrible policy decision. This will hurt the economy.",
                "Very disappointing statement. No real solutions proposed.",
                "Poor leadership on this issue. We need better direction."
            ]
        },
        'entertainment': {
            'positive': [
                "Fantastic movie! The plot was engaging and well-executed.",
                "Loved this new album. Every song is a masterpiece.",
                "Brilliant TV show. Can't wait for the next episode!"
            ],
            'negative': [
                "Boring movie. Predictable plot and bad acting.",
                "Disappointing album. Nothing memorable or original.",
                "Terrible show. Waste of time watching this."
            ]
        }
    }
    
    data = []
    
    for _ in range(num_samples):
        # Random topic
        topic = random.choice(topics)
        topic_id = topics.index(topic)
        
        # Random sentiment
        sentiment = random.choice(['positive', 'negative'])
        sentiment_id = 1 if sentiment == 'positive' else 0
        
        # Random text
        text = random.choice(templates[topic][sentiment])
        
        # Add some variation
        if random.random() > 0.5:
            text = text + " " + random.choice(templates[topic][sentiment])
        
        # Determine length category
        length = len(text.split())
        if length < 10:
            length_id = 0  # short
        elif length < 20:
            length_id = 1  # medium
        else:
            length_id = 2  # long
        
        data.append({
            'text': text,
            'sentiment_label': sentiment_id,
            'topic_label': topic_id,
            'length_label': length_id,
        })
    
    return Dataset.from_list(data)

# Create datasets
print("📦 Creating multi-task dataset...\n")
train_dataset = create_multitask_dataset(1000)
test_dataset = create_multitask_dataset(200)

print("✅ Dataset created!")
print(f"\n📊 Training samples: {len(train_dataset)}")
print(f"📊 Test samples: {len(test_dataset)}")

# Show example
print("\n🔍 Example data point:")
example = train_dataset[0]
print(f"\nText: {example['text']}")
print(f"Sentiment: {example['sentiment_label']} (0=negative, 1=positive)")
print(f"Topic: {example['topic_label']} (0=tech, 1=sports, 2=politics, 3=entertainment)")
print(f"Length: {example['length_label']} (0=short, 1=medium, 2=long)")

## Step 3: Create Multi-Task Trainer

This is where the magic happens - we train on all tasks!

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

class MultiTaskTrainer:
    """Custom trainer for multi-task learning"""
    
    def __init__(self, model, tokenizer, train_dataset, test_dataset):
        self.model = model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
        # Task names
        self.tasks = ['sentiment', 'topic', 'length']
        
        # Move model to device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        print(f"🖥️ Using device: {self.device}")
    
    def train(self, num_epochs=3, batch_size=16, learning_rate=3e-5):
        """Train on all tasks simultaneously"""
        
        # Optimizer
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)
        
        # Loss function
        criterion = nn.CrossEntropyLoss()
        
        print(f"\n🚀 Starting multi-task training...")
        print(f"   Epochs: {num_epochs}")
        print(f"   Batch size: {batch_size}")
        print(f"   Learning rate: {learning_rate}\n")
        
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            task_losses = {task: 0 for task in self.tasks}
            
            # Create batches
            num_batches = len(self.train_dataset) // batch_size
            
            progress_bar = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}")
            
            for batch_idx in progress_bar:
                # Get batch
                start_idx = batch_idx * batch_size
                end_idx = start_idx + batch_size
                batch = self.train_dataset[start_idx:end_idx]
                
                # Tokenize
                inputs = self.tokenizer(
                    batch['text'],
                    padding=True,
                    truncation=True,
                    max_length=128,
                    return_tensors='pt'
                ).to(self.device)
                
                # Train on each task
                batch_loss = 0
                
                for task in self.tasks:
                    # Get predictions for this task
                    logits = self.model(
                        inputs['input_ids'],
                        inputs['attention_mask'],
                        task_name=task
                    )
                    
                    # Get labels for this task
                    labels = torch.tensor(batch[f'{task}_label']).to(self.device)
                    
                    # Calculate loss
                    loss = criterion(logits, labels)
                    batch_loss += loss
                    task_losses[task] += loss.item()
                
                # Average loss across tasks
                batch_loss = batch_loss / len(self.tasks)
                
                # Backprop
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()
                
                total_loss += batch_loss.item()
                
                # Update progress bar
                progress_bar.set_postfix({'loss': f'{batch_loss.item():.4f}'})
            
            # Epoch summary
            avg_loss = total_loss / num_batches
            print(f"\n   📊 Epoch {epoch+1} completed!")
            print(f"   📉 Average loss: {avg_loss:.4f}")
            
            for task in self.tasks:
                task_avg = task_losses[task] / num_batches
                print(f"   📌 {task} loss: {task_avg:.4f}")
            
            # Evaluate after each epoch
            print()
            self.evaluate()
            print()
    
    def evaluate(self):
        """Evaluate on all tasks"""
        self.model.eval()
        
        results = {task: {'correct': 0, 'total': 0} for task in self.tasks}
        
        with torch.no_grad():
            for example in self.test_dataset:
                # Tokenize
                inputs = self.tokenizer(
                    example['text'],
                    padding=True,
                    truncation=True,
                    max_length=128,
                    return_tensors='pt'
                ).to(self.device)
                
                # Evaluate each task
                for task in self.tasks:
                    logits = self.model(
                        inputs['input_ids'],
                        inputs['attention_mask'],
                        task_name=task
                    )
                    
                    pred = logits.argmax(dim=-1).item()
                    true_label = example[f'{task}_label']
                    
                    if pred == true_label:
                        results[task]['correct'] += 1
                    results[task]['total'] += 1
        
        # Print results
        print("   🎯 Evaluation Results:")
        for task in self.tasks:
            accuracy = results[task]['correct'] / results[task]['total']
            print(f"   {task.capitalize()}: {accuracy:.3f} ({results[task]['correct']}/{results[task]['total']})")
        
        # Overall average
        avg_accuracy = np.mean([results[task]['correct']/results[task]['total'] for task in self.tasks])
        print(f"   ⭐ Average: {avg_accuracy:.3f}")

# Create trainer
trainer = MultiTaskTrainer(model, tokenizer, train_dataset, test_dataset)

print("✅ Trainer ready!")

## Step 4: Train the Multi-Task Model!

In [ ]:
# Train!
trainer.train(num_epochs=3, batch_size=16, learning_rate=3e-5)

## 🎉 Look at That!

You just trained ONE model to do THREE different tasks!

**Notice:**
- The model learned all three tasks simultaneously
- Tasks helped each other (shared encoder)
- Single training loop, multiple outputs
- More efficient than training 3 separate models!

Let's test it out:

## Step 5: Test Our Multi-Task Model

In [ ]:
def predict_all_tasks(text):
    """Get predictions from all tasks for a given text"""
    
    model.eval()
    
    # Tokenize
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    ).to(trainer.device)
    
    # Get predictions from all tasks
    predictions = {}
    
    with torch.no_grad():
        for task in trainer.tasks:
            logits = model(
                inputs['input_ids'],
                inputs['attention_mask'],
                task_name=task
            )
            pred_id = logits.argmax(dim=-1).item()
            predictions[task] = pred_id
    
    return predictions

# Test examples
test_texts = [
    "This new smartphone is absolutely amazing! Best tech purchase ever!",
    "Terrible game today. The team played horribly.",
    "The new policy is excellent. This will help many people.",
    "Loved this movie! Fantastic plot and great acting.",
]

label_names = {
    'sentiment': ['Negative', 'Positive'],
    'topic': ['Tech', 'Sports', 'Politics', 'Entertainment'],
    'length': ['Short', 'Medium', 'Long']
}

print("🧪 Testing our multi-task model!\n")
print("="*80)

for i, text in enumerate(test_texts, 1):
    print(f"\n📝 Example {i}:")
    print(f"   Text: \"{text}\"\n")
    
    preds = predict_all_tasks(text)
    
    print("   Predictions:")
    for task, pred_id in preds.items():
        label = label_names[task][pred_id]
        print(f"   • {task.capitalize()}: {label}")
    
    print("   " + "-"*60)

print("\n🎉 One model, three predictions! Amazing!")

## 🎓 Advanced Multi-Task Strategies

### 1. **Task Weighting**

Not all tasks are equally important. Weight them differently:

```python
task_weights = {
    'sentiment': 2.0,  # Most important
    'topic': 1.0,      # Medium
    'length': 0.5,     # Less important
}

# In training loop:
weighted_loss = sum(task_weights[task] * loss for task, loss in losses.items())
```

### 2. **Task Sampling**

Instead of training on ALL tasks each batch, sample tasks:

```python
# Each batch, randomly pick 1-2 tasks to train
active_tasks = random.sample(all_tasks, k=2)
loss = sum(losses[task] for task in active_tasks)
```

### 3. **Gradual Task Introduction**

Start with one task, gradually add more:

```python
if epoch < 2:
    tasks = ['sentiment']  # Start with main task
elif epoch < 4:
    tasks = ['sentiment', 'topic']  # Add second task
else:
    tasks = ['sentiment', 'topic', 'length']  # All tasks
```

### 4. **Task-Specific Learning Rates**

Different tasks might need different learning rates:

```python
optimizer = AdamW([
    {'params': model.encoder.parameters(), 'lr': 2e-5},
    {'params': model.task_heads['sentiment'].parameters(), 'lr': 5e-5},
    {'params': model.task_heads['topic'].parameters(), 'lr': 3e-5},
])
```

## 💡 When to Use Multi-Task Learning

**✅ GOOD Use Cases:**

1. **Related Tasks**
   - Sentiment + Topic classification ✅
   - NER + POS tagging ✅
   - Q&A + Summarization ✅

2. **Limited Data for Some Tasks**
   - Lots of data for Task A
   - Little data for Task B
   - MTL helps Task B learn from Task A!

3. **Production Efficiency**
   - Deploy one model instead of many
   - Save memory and costs

**❌ BAD Use Cases:**

1. **Completely Unrelated Tasks**
   - Image classification + Text summarization ❌
   - Tasks that don't share knowledge

2. **Conflicting Objectives**
   - Tasks that require opposite features
   - May hurt each other's performance

3. **One Task Has Plenty of Data**
   - If Task A has 1M examples and Task B has 100
   - Task A might dominate training
   - Use task weighting to fix this!

## 🔧 Practical Tips

### 1. **Monitor Each Task Separately**
Don't just look at average loss - track each task's performance!

### 2. **Balance Your Batches**
Make sure each batch has examples from all tasks (or sample fairly)

### 3. **Shared vs Task-Specific**
- Encoder = shared (learns general features)
- Heads = task-specific (learns task details)
- You can also add task-specific middle layers!

### 4. **Learning Rate**
MTL often works better with slightly LOWER learning rates
- Try 2e-5 instead of 3e-5

### 5. **Evaluation**
Evaluate EACH task on its own test set
- Don't just look at combined metrics

### 6. **Debugging**
If one task is hurting another:
- Try task weighting
- Try task sampling
- Maybe train separately after all!

## 🎉 You're Now a Multi-Task Learning Expert!

You just learned:
- ✅ How to build multi-task architectures
- ✅ How to train on multiple tasks simultaneously
- ✅ When to use MTL (and when not to)
- ✅ Advanced strategies (weighting, sampling, etc.)
- ✅ How tasks can help each other learn

**Real-World Impact:**
- Google's BERT: Multi-task training
- T5: Trained on 20+ tasks!
- GPT models: Multi-task learners

Multi-task learning is how modern AI gets so powerful!

Pretty cool, right? 🚀

---

**Next up:** Custom Loss Functions - Get really creative! 🎲

## 📚 Further Reading

**Papers:**
- [An Overview of Multi-Task Learning in Deep Neural Networks](https://arxiv.org/abs/1706.05098)
- [BERT: Pre-training of Deep Bidirectional Transformers](https://arxiv.org/abs/1810.04805)
- [T5: Exploring the Limits of Transfer Learning](https://arxiv.org/abs/1910.10683)

**Resources:**
- [HuggingFace Multi-Task Learning Examples](https://huggingface.co/docs/transformers/tasks_explained#multi-task-learning)
- [Fast.ai MTL Tutorial](https://www.fast.ai)

Now go train some multi-talented models! 💪